# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = PyPDFLoader("intelligent_scissor.pdf")
documents = loader.load_and_split(text_splitter)
documents

[Document(metadata={'source': 'intelligent_scissor.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/220720964\nIntelligent scissors for image composition\nConference Paper · January 1995\nDOI: 10.1145/218380.218442\xa0·\xa0Source: DBLP\nCITATIONS\n812\nREADS\n2,545\n2 authors, including:\nEric N. Mortensen\nLucidyne Technologies, Inc.\n34 PUBLICATIONS\xa0\xa0\xa03,134 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Eric N. Mortensen on 01 June 2014.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'intelligent_scissor.pdf', 'page': 1}, page_content='Abstract\nWe present a new, interactive tool called Intelligent Scissors\nwhich we use for image segmentation and composition.  Fully auto-\nmated segmentation is an unsolved problem, while manual tracing\nis inaccurate and laboriously unacceptable.  However, Intelligent

## Load the Content in a Vector Store

In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

c:\Users\zaodu\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [6]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,"See discussions, stats, and author profiles fo..."
1,"Abstract\nWe present a new, interactive tool c..."
2,"lowed, rather than simply the strongest edge i..."
3,"manipulation techniques, has also been used to..."
4,control the segmentation process.\nThis paper ...
5,"tions, use an interactively selected seed poin..."
6,boundary will look like when the rough approxi...
7,of freedom within a window about the two-dimen...
8,permission and/or a fee. \n©1995 ACM-0-89791-...
9,The most important difference between previous...


We can now create a Knowledge Base using the DataFrame we created before.

In [9]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

## Generate the Test Set

In [11]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about the intelligent scissor paper",
)

Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

2024-11-17 13:50:31,051 pid:3908 MainThread giskard.rag  ERROR    Encountered error in question generation: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}. Skipping.
2024-11-17 13:50:31,051 pid:3908 MainThread giskard.rag  ERROR    Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Traceback (most recent call last):
  File "c:\Users\zaodu\AppData\Local\Programs\Python\Python312\Lib\site-packages\giskard\rag\question_generators\base.py", line 80, in generate_questions
    

Let's display a few samples from the test set.

In [12]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What are the limitations of the training based on learned edge characteristics?
Reference answer: Training is most effective for those objects with edge properties that are relatively consistent along the object boundary. Training can be counter-productive for objects with sudden and/or dramatic changes in edge features. However, training can be turned on and off interactively throughout the definition of an object boundary.
Reference context:
Document 32: precomputed feature maps along the closestt pixels of the edge seg-
ment and increments the feature histogram element by the corre-
sponding pixel weight to generate a histogram for each feature
involved in training.
After sampling and smoothing, each feature histogram is then
scaled and inverted (by subtracting the scaled histogram values
from its maximum value) to create the feature cost map needed to
convert feature values to trained cost functions.
Since training is based on learned edge characteristics from the
most 

Let's now save the test set to a file:

In [13]:
testset.save("RAG-test-set.jsonl")

## Prepare the Prompt Template

In [14]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [17]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("what is the intelligent scissor paper about?")

[Document(metadata={'source': 'intelligent_scissor.pdf', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/220720964\nIntelligent scissors for image composition\nConference Paper · January 1995\nDOI: 10.1145/218380.218442\xa0·\xa0Source: DBLP\nCITATIONS\n812\nREADS\n2,545\n2 authors, including:\nEric N. Mortensen\nLucidyne Technologies, Inc.\n34 PUBLICATIONS\xa0\xa0\xa03,134 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Eric N. Mortensen on 01 June 2014.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={'source': 'intelligent_scissor.pdf', 'page': 6}, page_content='2-D extension to previous optimal edge tracking methods rather\nthan an improvement on active contours.\n4. Image Composition with Intelligent Scissors\nAs mentioned, composition artists need an intelligent, interactive\ntool to facilitate image component boundary deﬁnit

We can now create our chain.

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [19]:
chain.invoke({"question": "What is the intelligent scissor paper about?"})

'The intelligent scissor paper is about image composition and segmentation using a new interactive tool called Intelligent Scissors.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [20]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [23]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/53 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/53 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [24]:
display(report)

Loading BokehJS ...

In [ ]:
report.to_html("report.html")

We can display the correctness results organized by question type.

In [25]:
report.correctness_by_question_type()

,correctness
question_type,
complex,1.0
conversational,0.0
distracting element,0.3
double,0.5
simple,0.8
situational,0.7


We can also display the specific failures.

In [26]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
336e5430-3023-42cf-a975-4b9da8b53c25,What are the limitations of the training based...,Training is most effective for those objects w...,Document 32: precomputed feature maps along th...,[],"{'question_type': 'simple', 'seed_document_id'...",The limitations of the training based on learn...,False,The agent's answer did not mention that traini...
2590f4db-5a36-4e3d-86dc-10f1e59ae602,What does preprocessing require for color imag...,Preprocessing requires 36 convolutions for col...,Document 44: times (for a trained user) for ea...,[],"{'question_type': 'simple', 'seed_document_id'...",I don't know.,False,The agent stated that it doesn't know the answ...
e2ebb642-6503-4a09-a071-2574c81f2089,What was the objective of the study conducted ...,The study with eight untrained users was condu...,Document 44: times (for a trained user) for ea...,[],"{'question_type': 'distracting element', 'seed...",The objective of the study conducted with eigh...,False,The agent stated that the objective of the stu...
a5b51156-fde2-4faf-86f3-9b983162591c,What is the copyright year and fee for the mat...,The copyright year is 1995 and the fee is $3.50.,Document 8: permission and/or a fee. \n©1995 ...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent stated that it doesn't know the copy...
4a9f105f-5bd2-4feb-b123-0f54187c3708,In the context of the Live-Wire 2-D DP graph s...,The gradient direction adds a smoothness const...,"Document 13: imum gradient at unity,fG ( q ) i...",[],"{'question_type': 'distracting element', 'seed...",The gradient direction function ensures smooth...,False,The agent's answer was not specific enough. Th...
c66dc718-27b0-4289-88b5-f0e0fae21b25,"Considering the context of image composition, ...",The five key methods are: (1) making use of th...,Document 48: (1) making use of the weighted ze...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,The agent stated that it doesn't know the answ...
d097df72-9b9b-44ed-969c-55123f98df42,In the context of image composition using Inte...,Subpixel accuracy can be obtained by exploitin...,Document 39: efﬁciency.\n1. Similar in concept...,[],"{'question_type': 'distracting element', 'seed...",In the context of image composition using Inte...,False,The agent provided an answer about using live-...
6a48b2d0-45a6-435b-a067-2180ea37150b,Considering the wavefront of active points tha...,Intelligent Scissors require less time and eff...,"Document 1: Abstract\nWe present a new, intera...",[],"{'question_type': 'distracting element', 'seed...",The key advantage of Intelligent Scissors over...,False,The agent stated that the advantage of Intelli...
9db4f3f0-a35f-45bf-834d-0741d1ea4a83,Can you identify the authors of the Intelligen...,The authors of the Intelligent Scissors for Im...,Document 7: of freedom within a window about t...,[],"{'question_type': 'distracting element', 'seed...","Yes, the authors of the Intelligent Scissors f...",False,The agent incorrectly stated that the authors ...


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [28]:
from giskard.rag import QATestset

testset = QATestset.load("RAG-test-set.jsonl")

Create a Test Suite from the test set.

In [29]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [30]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [31]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

2024-11-17 14:09:24,203 pid:3908 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [32]:
test_suite_results = test_suite.run(model=giskard_model)

2024-11-17 14:09:28,728 pid:3908 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-11-17 14:09:33,388 pid:3908 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (53, 5) executed in 0:00:04.659229
2024-11-17 14:11:00,798 pid:3908 MainThread root         ERROR    An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "c:\Users\zaodu\AppData\Local\Programs\Python\Python312\Lib\site-packages\giskard\core\suite.py", line 522, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\zaodu\AppData\Local\Programs\Python\Python312\Lib\site-packages\giskard\registry\giskard_test.py", line 195, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

We can display the results.

In [33]:
display(test_suite_results)

<TestSuiteResult (failed)>

## Integrating with Pytest

In [35]:
import ipytest

We can now integrate our test suite with Pytest.

In [37]:
# %%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()